In [1]:
%%bash
text2workspace.py datacard_shape_blackbox_1000.txt -m 3500 -o workspace_blackbox_1000.root

Channel signal_region has bin 59 fill in data but empty in all backgrounds
Channel signal_region has bin 60 fill in data but empty in all backgrounds
Channel signal_region has bin 68 fill in data but empty in all backgrounds


In [2]:
%%bash
combine -M AsymptoticLimits workspace_blackbox_1000.root -m 3500

 <<< Combine >>> 
>>> method used is AsymptoticLimits
>>> random number generator seed is 123456
SimNLL created with 1 channels, 0 generic constraints, 3 fast gaussian constraints, 0 fast poisson constraints, 0 fast group constraints, 
SimNLL created with 1 channels, 0 generic constraints, 3 fast gaussian constraints, 0 fast poisson constraints, 0 fast group constraints, 
SimNLL created with 1 channels, 0 generic constraints, 3 fast gaussian constraints, 0 fast poisson constraints, 0 fast group constraints, 

 -- AsymptoticLimits ( CLs ) --
Observed Limit: r < 0.4507
Expected  2.5%: r < 0.0063
Expected 16.0%: r < 0.0095
Expected 50.0%: r < 0.0156
Expected 84.0%: r < 0.0241
Expected 97.5%: r < 0.0340

Done in 0.00 min (cpu), 0.00 min (real)


Warning! -- You haven't picked default values for the Parameters of Interest (either with --expectSignal or --setParameters) for generating toys. Combine will use the 'B-only' ModelConfig to generate, which may lead to undesired behaviour if not using the default Physics Model


In [3]:
%%bash
combine -M Significance workspace_blackbox_1000.root -m 3500

 <<< Combine >>> 
>>> method used is Significance
>>> random number generator seed is 123456

 -- Significance -- 
Significance: 28.8277
Done in 0.00 min (cpu), 0.00 min (real)


Warning! -- You haven't picked default values for the Parameters of Interest (either with --expectSignal or --setParameters) for generating toys. Combine will use the 'B-only' ModelConfig to generate, which may lead to undesired behaviour if not using the default Physics Model


In [4]:
%%bash
combine -M FitDiagnostics workspace_blackbox_1000.root -m 3500 --saveShapes --saveWithUncertainties

 <<< Combine >>> 
>>> method used is FitDiagnostics
>>> random number generator seed is 123456
SimNLL created with 0 channels, 0 generic constraints, 3 fast gaussian constraints, 0 fast poisson constraints, 0 fast group constraints, 
SimNLL created with 1 channels, 0 generic constraints, 3 fast gaussian constraints, 0 fast poisson constraints, 0 fast group constraints, 

 --- FitDiagnostics ---
Best fit r: 0.449501  -0.0351455/+0.0378058  (68% CL)
Done in 0.01 min (cpu), 0.01 min (real)


Warning! -- You haven't picked default values for the Parameters of Interest (either with --expectSignal or --setParameters) for generating toys. Combine will use the 'B-only' ModelConfig to generate, which may lead to undesired behaviour if not using the default Physics Model


In [5]:
import ROOT
fin = ROOT.TFile('fitDiagnostics.root','read')
#fin.ls()
fin.cd('shapes_fit_b/signal_region')
#fin.cd('signal_region')
fin.ls()
fin.Close()

Welcome to JupyROOT 6.12/07


In [5]:
def fix(iH0,iH1):
    lTHist = ROOT.TH1F(iH1.GetName(),iH1.GetName(),iH0.GetNbinsX(),iH0.GetXaxis().GetXmin(),iH0.GetXaxis().GetXmax())
    for i0 in range(0,iH0.GetNbinsX()+1):
        print(i0,iH1.GetBinContent(i0))
        lTHist.SetBinContent(i0,iH1.GetBinContent(i0))
        lTHist.SetBinError(i0,iH1.GetBinError(i0))
    lTHist.SetFillStyle(iH1.GetFillStyle())
    lTHist.SetFillColor(iH1.GetFillColor())
    lTHist.SetLineColor(iH1.GetLineColor())
    lTHist.SetLineStyle(iH1.GetLineStyle())
    return lTHist

In [6]:
def fix_graph(iH0,iH1):
    lTGraph = ROOT.TGraphAsymmErrors(iH1.GetN())
    a = iH1.GetY()
    print(a[0])
    for i0 in range(0,iH0.GetNbinsX()):
        print(i0,iH0.GetXaxis().GetBinCenter(i0+1),a[i0])
        lTGraph.SetPoint(i0,iH0.GetXaxis().GetBinCenter(i0+1),a[i0])
        lTGraph.SetPointError(i0,ROOT.NULL,ROOT.NULL, iH1.GetErrorYlow(i0), iH1.GetErrorYhigh(i0))
    lTGraph.SetFillStyle(iH1.GetFillStyle())
    lTGraph.SetFillColor(iH1.GetFillColor())
    lTGraph.SetLineColor(iH1.GetLineColor())
    lTGraph.SetLineStyle(iH1.GetLineStyle())
    return lTGraph

In [8]:
import sys,os
sys.path.append('/data/t3home000/spark/QUASAR')

import plotting as plot
import ROOT

ROOT.PyConfig.IgnoreCommandLineOptions = True
ROOT.gROOT.SetBatch(ROOT.kTRUE)

plot.ModTDRStyle()
canvas = ROOT.TCanvas()

fin = ROOT.TFile('fitDiagnostics.root')

first_dir = 'shapes_fit_b'
second_dir = 'signal_region'

h_bkg = fin.Get(first_dir + '/' + second_dir + '/total_background')
h_sig = fin.Get('shapes_fit_s' + '/' + second_dir + '/total_signal')
#h_tot = fin.Get('shapes_fit_s' + '/' + second_dir + '/total')

h_dat = fin.Get(first_dir + '/' + second_dir + '/data')  # This is a TGraphAsymmErrors, not a TH1F

start, end = (2800,8000)

postfit_obs = ROOT.TH1F('postfit_obs','postfit_obs',(end-start)/100,start,end)
postfit_bkg = ROOT.TH1F('postfit_bkg','postfit_bkg',(end-start)/100,start,end)
postfit_sig = ROOT.TH1F('postfit_sig','postfit_sig',(end-start)/100,start,end)


h_bkg = fix(postfit_bkg,h_bkg)
h_sig = fix(postfit_sig,h_sig)
h_dat = fix_graph(postfit_obs,h_dat)
h_tot = fix_graph(postfit_bkg,h_dat)



h_bkg.SetFillColor(ROOT.TColor.GetColor(100, 192, 232))
h_bkg.GetXaxis().SetTitle("m_{JJ} [GeV]")
h_bkg.Draw('HIST')

h_err = h_bkg.Clone()
h_err.SetFillColorAlpha(12, 0.3)  # Set grey colour (12) and alpha (0.3)
h_err.SetMarkerSize(0)
h_err.Draw('E2SAME')

h_sig.SetLineColor(ROOT.kRed)
h_sig.Draw('HISTSAME')

h_dat.Draw('PSAME')

h_tot.SetMarkerColor(ROOT.kRed)
#h_tot.Draw('PSAME')

h_bkg.SetMaximum(h_bkg.GetMaximum() * 1.4)

legend = ROOT.TLegend(0.60, 0.70, 0.90, 0.91, '', 'NBNDC')
legend.AddEntry(h_bkg, 'Background', 'F')
legend.AddEntry(h_sig, 'Signal', 'L')
legend.AddEntry(h_err, 'Background uncertainty', 'F')
legend.Draw()

canvas.SaveAs('plot-bb-1000_fullrange.pdf')
canvas.SaveAs('plot-bb-1000_fullrange.png')

(0, 0.0)
(1, 4.777049980475567e-06)
(2, 4.777049980475567e-06)
(3, 1.2267719507217407)
(4, 1.2267719507217407)
(5, 4.907087802886963)
(6, 11.040947914123535)
(7, 25.7622127532959)
(8, 65.0189208984375)
(9, 103.0488510131836)
(10, 150.89295959472656)
(11, 181.56224060058594)
(12, 193.82997131347656)
(13, 202.41738891601562)
(14, 186.46934509277344)
(15, 230.6331329345703)
(16, 262.5292053222656)
(17, 263.7559814453125)
(18, 213.4583282470703)
(19, 257.6221008300781)
(20, 255.1685791015625)
(21, 226.95281982421875)
(22, 198.737060546875)
(23, 176.65516662597656)
(24, 201.19061279296875)
(25, 139.8520050048828)
(26, 142.30555725097656)
(27, 139.8520050048828)
(28, 118.99688720703125)
(29, 100.59530639648438)
(30, 77.28663635253906)
(31, 94.46144104003906)
(32, 69.9260025024414)
(33, 56.43151092529297)
(34, 51.5244255065918)
(35, 61.338600158691406)
(36, 60.1118278503418)
(37, 30.669300079345703)
(38, 23.30866813659668)
(39, 29.44252586364746)
(40, 24.535438537597656)
(41, 18.4015789031982

Info in <TCanvas::Print>: pdf file plot-bb-1000_fullrange.pdf has been created
Info in <TCanvas::Print>: png file plot-bb-1000_fullrange.png has been created


In [260]:
import ROOT as rt

In [261]:
fd = ROOT.TFile.Open("fitDiagnostics.root",'read')

In [262]:
fd.ls()

TFile**		fitDiagnostics.root	
 TFile*		fitDiagnostics.root	
  KEY: TDirectoryFile	shapes_prefit;1	shapes_prefit
  KEY: RooArgSet	norm_prefit;1	Set of RooAbsArg objects
  KEY: TProcessID	ProcessID0;1	61ca9cc8-9b2d-11ea-9088-51700412beef
  KEY: RooFitResult	nuisances_prefit_res;1	
  KEY: RooArgSet	nuisances_prefit;1	Set of RooAbsArg objects
  KEY: RooFitResult	fit_b;1	
  KEY: TDirectoryFile	shapes_fit_b;1	shapes_fit_b
  KEY: RooArgSet	norm_fit_b;1	Set of RooAbsArg objects
  KEY: RooFitResult	fit_s;1	
  KEY: TDirectoryFile	shapes_fit_s;1	shapes_fit_s
  KEY: RooArgSet	norm_fit_s;1	Set of RooAbsArg objects
  KEY: TTree	tree_fit_sb;1	tree_fit_sb
  KEY: TTree	tree_fit_b;1	tree_fit_b
  KEY: TTree	tree_prefit;1	tree_prefit


In [263]:
obj = fd.Get('fit_s')

In [264]:
obj.Print()


  RooFitResult: minimized FCN value: -5570.66, estimated distance to minimum: 7.94396e-06
                covariance matrix quality: Full, accurate covariance matrix
                Status : MINIMIZE=0 HESSE=0 

    Floating Parameter    FinalValue +/-  Error   
  --------------------  --------------------------
                  lumi    4.5836e-01 +/-  2.27e-01
                     r    7.0428e-02 +/-  1.65e-02
                 scale    1.8976e-01 +/-  1.00e-01
                 smear   -1.1126e+00 +/-  1.72e-01



In [265]:
fd.Close()